In [1]:
#server.py
import socket #socket 프로그래밍을 위한 
import json #json 사용을 위한 라이브러리
import threading #쓰레드를 사용해서 백그라운드로 실행할거임

HOST = '0.0.0.0'
PORT = 12008
BUFFER_SIZE = 2048

# 메시지 처리 함수
def process_message(username, message, option_raw): #서버로 부터 받아온 JSON 데이터를 처리하기 위한 함수
    try: 
        if "." in str(option_raw):
            raise ValueError("float")
        option = int(option_raw)
        # 예외에 걸리면 상태 코드랑 에러 메시지 반환
        if option < 1: #1 미만이라면
            return {'status': 401, 'error': 'option input error: integer less than 1'}, False
        elif option > 4: # 4 초과라면
            return {'status': 402, 'error': 'option input error: greater than 4'}, False

    except ValueError as ve:
        if str(ve) == "float": #실수라면
            return {'status': 403, 'error': 'option input error: non-integer'}, False
        else: #숫자가 아니라면
            return {'status': 405, 'error': 'option input error: non-numeric input'}, False

    # 정상 입력인 경우 Before / After 메시지 출력
    print(f"Before: [{username}]: {message}")

    #옵션처리
    if option == 1:  #1이면 그대로
        result = message
    elif option == 2: #2이면 대문자로 변환
        result = message.upper()
    elif option == 3: #3이면 소문자로 변환
        result = message.lower()

    print(f"After:  [{username}]: {result}") #결과를 서버에 출력
    return {'status': 200, 'message': f'[{username}]: {result}'}, False #성공하면 성공 메세지랑 상태 200 반환

# 서버 시작 함수
def start_server():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket: #서버 연결
        server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        server_socket.bind((HOST, PORT)) #IP/포트를 서버에 할당
        server_socket.listen(5) # 클라이언트의 연결 요청을 받을 수 있도록 리스닝 상태로 설정 (최대 5개까지 대기 가능)
        print(f"[SERVER] Listening on port {PORT}...")

        while True:
            client_socket, addr = server_socket.accept()
            with client_socket:
                data = client_socket.recv(BUFFER_SIZE).decode() #클라이언트한테 데이터 받음(JSON)
                try:
                    request = json.loads(data) 
                    username = request.get('user') #사용자
                    message = request.get('message') #메시지
                    option = request.get('echooption') #옵션

                    response, _ = process_message(username, message, option) #처리
                    client_socket.sendall(json.dumps(response).encode()) #클라이언트로 전달

                    print(f"[RECV] {request}") #확인용 받은 정보 출력
                    print(f"[SEND] {response}") #확인용 전달한 정보 출력
                    print("-" * 40)

                except Exception as e: #예상하지 못한 오류 처리
                    error_response = json.dumps({"status": 500, "error": "Invalid request format"})
                    client_socket.sendall(error_response.encode())
                    print(f"[ERROR] {e}")

# 백그라운드 스레드로 실행
server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()

[SERVER] Listening on port 12008...


Before: [김현진]: hihi
After:  [김현진]: HIHI
[RECV] {'user': '김현진', 'echooption': '2', 'message': 'hihi'}
[SEND] {'status': 200, 'message': '[김현진]: HIHI'}
----------------------------------------
Before: [김현진]: QWER
After:  [김현진]: qwer
[RECV] {'user': '김현진', 'echooption': '3', 'message': 'QWER'}
[SEND] {'status': 200, 'message': '[김현진]: qwer'}
----------------------------------------
Before: [김현진]: asdf
After:  [김현진]: asdf
[RECV] {'user': '김현진', 'echooption': '1', 'message': 'asdf'}
[SEND] {'status': 200, 'message': '[김현진]: asdf'}
----------------------------------------
[RECV] {'user': '김현진', 'echooption': '1.1', 'message': 'asdf'}
[SEND] {'status': 403, 'error': 'option input error: non-integer'}
----------------------------------------
Before: [김현진]: hihi
After:  [김현진]: HIHI
[RECV] {'user': '김현진', 'echooption': '2', 'message': 'hihi'}
[SEND] {'status': 200, 'message': '[김현진]: HIHI'}
----------------------------------------
[RECV] {'user': '채주혁', 'echooption': '0', 'message': 'ㅁㄴㅇㄹ'}
[SEN